In [1]:
import numpy as np
np.random.seed(2002)

In [3]:
data = np.load(r"D:\InterviewExercises\data\mnist.npz")
x_train = data['x_train']
y_train = data['y_train']
x_test = data['x_test']
y_test = data['y_test']

# chuẩn hóa
x_train = x_train / 255.0
x_test = x_test / 255.0

# in ra kích thước
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [70]:
# Chuyển đổi dữ liệu từ 2D (28x28) thành 1D (784)
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# in ra kích thước
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [72]:
x_train = np.hstack([np.ones((x_train.shape[0], 1)), x_train])
x_test = np.hstack([np.ones((x_test.shape[0], 1)), x_test])
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(60000, 786) (60000,)
(10000, 786) (10000,)


In [73]:
def generate_triplets(X, y):
    triplets = []
    for anchor_idx in range(len(y)):
        anchor_label = y[anchor_idx]
        positive_idx = np.random.choice(np.where(y == anchor_label)[0])
        negative_idx = np.random.choice(np.where(y != anchor_label)[0])
        triplets.append((X[anchor_idx], X[positive_idx], X[negative_idx]))
    return np.array(triplets)
x_train_triplets = generate_triplets(x_train, y_train)

In [74]:
print(x_train_triplets[0][0].shape)
print(x_train_triplets[0][1].shape)
print(x_train_triplets[0][2].shape)

(786,)
(786,)
(786,)


In [76]:
def compute_gradients(W, x, reg, margin=1.0):
    '''
    Input:
      W: Một mảng có kích thước (D, C) chứa các trọng số (weights).
      X: Một mảng có kích thước (N, 3, D) chứa một lô nhỏ dữ liệu (mini-batch of data).
      reg: (float) cường độ điều chuẩn (regularization strength).
    Output:
      Mất mát (loss) dưới dạng số thực (float).
      Gradient đối với trọng số W; một mảng có kích thước giống như W.
    '''

    # print(x[:, 0, :].shape)
    # print(x[:, 1, :].shape)
    # print(x[:, 2, :].shape)
    # print(W.shape)

    loss = 0.0
    dW = np.zeros_like(W)
    N = x.shape[0]
    anchor, positive, negative = x[:, 0, :], x[:, 1, :], x[:, 2, :]

    scores_anchor = anchor.dot(W)
    scores_positive = positive.dot(W)
    scores_negative = negative.dot(W)

    pos_dist = np.sum((scores_anchor - scores_positive) ** 2, axis=1)
    neg_dist = np.sum((scores_anchor - scores_negative) ** 2, axis=1)

    # Tính triplet_loss
    loss = np.maximum(0, pos_dist - neg_dist + margin)
    avg_loss = np.mean(loss)

    # Tính gradient chỉ khi loss > 0
    mask = (loss > 0).astype(float)

    # Gradient của hàm mất mát theo điểm số
    d_pos_dist = 2 * (scores_anchor - scores_positive)
    d_neg_dist = 2 * (scores_anchor - scores_negative)

    # Tính gradient đối với điểm số
    dloss_danchor = (d_pos_dist - d_neg_dist) * mask[:, np.newaxis]
    dloss_dpositive = -d_pos_dist * mask[:, np.newaxis]
    dloss_dnegative = d_neg_dist * mask[:, np.newaxis]

    # Tính gradient đối với trọng số W
    dW = (anchor.T.dot(dloss_danchor) +
          positive.T.dot(dloss_dpositive) +
          negative.T.dot(dloss_dnegative)) / N

    # Thêm cường độ điều chuẩn
    dW += reg * W

    # Tính toán mất mát với điều chuẩn
    avg_loss += 0.5 * reg * np.sum(W * W)

    return avg_loss, dW

In [86]:
W = np.random.randn(786, 300) * 0.01

In [87]:
avg_loss, W = compute_gradients(W,x_train_triplets[0:32], 0 )

In [88]:
print(avg_loss)

0.5263285890127638


In [89]:
epochs = 20
batch_size = 64
learning_rate = 0.01
reg = 0.01

def update_weights(W, dW, learning_rate):
    W -= learning_rate * dW
    return W

num_train = x_train_triplets.shape[0]
num_batches = num_train // batch_size

for epoch in range(epochs):
    epoch_loss = 0.0

    # Chia dữ liệu thành các batch
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        x_batch = x_train_triplets[batch_start:batch_end]

        # Tính toán loss và gradient
        loss, dW = compute_gradients(W, x_batch, reg)
        epoch_loss += loss

        # Cập nhật trọng số
        W = update_weights(W, dW, learning_rate)

    # In ra mất mát trung bình của epoch
    epoch_loss /= num_batches
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss}')


Epoch 1/20, Loss: 0.8827232869920849
Epoch 2/20, Loss: 0.25975620806864125
Epoch 3/20, Loss: 0.22281421976558916
Epoch 4/20, Loss: 0.21112978721010517
Epoch 5/20, Loss: 0.204939326127308
Epoch 6/20, Loss: 0.20119423229397165
Epoch 7/20, Loss: 0.19873228415418973
Epoch 8/20, Loss: 0.19682254884223158
Epoch 9/20, Loss: 0.19553122588464586
Epoch 10/20, Loss: 0.1943048300496394
Epoch 11/20, Loss: 0.19365675111653416
Epoch 12/20, Loss: 0.19280051815695146
Epoch 13/20, Loss: 0.19242539366955644
Epoch 14/20, Loss: 0.19188356353846273
Epoch 15/20, Loss: 0.19157996256568488
Epoch 16/20, Loss: 0.19121840469059329
Epoch 17/20, Loss: 0.1911777778702912
Epoch 18/20, Loss: 0.1910326643027605
Epoch 19/20, Loss: 0.19059971574002565
Epoch 20/20, Loss: 0.19046726105621284


### Predict

In [90]:
def extract_features(W, X):
    return X.dot(W)

In [91]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

In [92]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # Tính khoảng cách từ x tới tất cả các điểm trong X_train
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        # Lấy k láng giềng gần nhất
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Lấy nhãn phổ biến nhất
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

In [ ]:
train_features = extract_features(W, x_train)
test_features = extract_features(W, x_test)

k = 3
knn = KNN(k=k)
knn.fit(train_features, y_train)
# Dự đoán trên tập kiểm tra
predictions = knn.predict(test_features)

In [ ]:
# Tính độ chính xác
accuracy = np.sum(predictions == y_test[0:1000]) / len(y_test[0:1000])
print(f'Dộ chính xác của mô hình KNN là: {accuracy:.4f}')